In [ ]:
import pandas as pd

from pathlib import Path

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
accuracies_file: Path = Path("../results/cnn_hog_accuracies.csv").resolve()

accuracies_df: pd.DataFrame = pd.read_csv(accuracies_file.as_posix())

accuracies_df["Mean Accuracy"] = accuracies_df[accuracies_df.columns[2:]].mean(axis=1)

dataset_mean_series: pd.Series = accuracies_df.groupby("Datasets")["Mean Accuracy"].mean().sort_values(ascending=False)

dataset_mean_series.head(6)

In [ ]:
for filename in dataset_mean_series.head(6).index:
    file: Path = Path(f"../outputs/{filename}").resolve()

    df: pd.DataFrame = pd.read_csv(file.as_posix())

    features_df: pd.DataFrame = df.iloc[:, 3:]
    metadata_df: pd.DataFrame = df.iloc[:, :3]
    animal_series: pd.Series = df.animal

    scaler: StandardScaler = StandardScaler()
    scaled_features = scaler.fit_transform(features_df)

    pca: PCA = PCA(n_components=10)
    pca_features = pca.fit_transform(scaled_features)

    pca_features_df: pd.DataFrame = pd.DataFrame(pca_features)
    pca_features_df.rename(lambda x: f"feature_{int(x) + 1}", axis="columns", inplace=True)

    pca_df: pd.DataFrame = pd.concat([metadata_df, pca_features_df], axis="columns")

    output_file: Path = Path(f"../outputs/pca_{filename}").resolve()
    output_file.parent.mkdir(parents=True, exist_ok=True)

    df.to_csv(output_file.as_posix(), index=False)